In [204]:
import pickle
import numpy as np
import glob
import pandas as pd
import torch

In [205]:
results_path = 'test_results.pkl'
image_detection_data_path = '../data/Potholes/Proposals_test/test_detection/*'

In [206]:
# result = [predicted, label, path]
with open(results_path, "rb") as f:
    results = pickle.load(f)

# list[pkl]
image_detection_data_list = glob.glob(image_detection_data_path)

In [207]:
results_df = pd.DataFrame(results, columns=['predicted', 'label', 'proposal_path'])

In [208]:
results_df['proposal_path'] = results_df['proposal_path'].apply(lambda x: x.split('/')[-1])

In [209]:
rows = []

for image_detection_data_pkl in image_detection_data_list:
    img_id = int(image_detection_data_pkl.split('-')[-1].split('_')[0])
    
    with open(image_detection_data_pkl, "rb") as f:
        image_detection_data = pickle.load(f)
    
    groundtruth = image_detection_data['groundtruth']
    proposals = image_detection_data['proposals']
    
    max_len = max(len(groundtruth), len(proposals))
    
    for i in range(max_len):
        gt = groundtruth[i] if i < len(groundtruth) else None
        prop = proposals[i] if i < len(proposals) else None
        rows.append({
            'img_id': img_id,
            'groundtruth': gt,
            'proposals': prop
        })

image_detection_df = pd.DataFrame(rows)

In [210]:
image_detection_df['proposals'] = image_detection_df['proposals'].apply(lambda x: x if x is not None else [None, None])

image_detection_df[['proposal_path', 'proposal_bbox']] = pd.DataFrame(
    image_detection_df['proposals'].tolist(),
    index=image_detection_df.index
)

image_detection_df.drop(columns='proposals', inplace=True)

In [211]:
def convert_groundtruth(x):
    if x is None:  # Handle None values
        return []
    return [val.item() if isinstance(val, torch.Tensor) else val for val in x]

image_detection_df['groundtruth'] = image_detection_df['groundtruth'].apply(convert_groundtruth)


In [212]:
# Merge the two DataFrames
merged_df = pd.merge(image_detection_df, results_df, on='proposal_path', how='inner')

In [213]:
merged_df.head(20)

,img_id,groundtruth,proposal_path,proposal_bbox,predicted,label
0,268,"[258.0, 195.0, 494.0, 414.0]",img-268_proposal_0.jpg,"[258, 191, 498, 407]",0.0,0
1,268,[],img-268_proposal_1.jpg,"[258, 199, 498, 406]",0.0,0
2,268,[],img-268_proposal_2.jpg,"[258, 201, 498, 406]",0.0,0
3,268,[],img-268_proposal_3.jpg,"[252, 194, 502, 407]",0.0,0
4,268,[],img-268_proposal_4.jpg,"[252, 188, 502, 407]",0.0,0
5,268,[],img-268_proposal_5.jpg,"[269, 201, 498, 406]",0.0,0
6,268,[],img-268_proposal_6.jpg,"[274, 191, 498, 407]",0.0,0
7,268,[],img-268_proposal_7.jpg,"[252, 194, 486, 397]",0.0,0
8,268,[],img-268_proposal_8.jpg,"[252, 175, 505, 407]",0.0,0
9,268,[],img-268_proposal_9.jpg,"[231, 188, 502, 407]",0.0,0


In [ ]:
import pandas as pd
import numpy as np

def calculate_iou(box1, box2):
    """
    Calculate IoU between two boxes
    boxes format: [x1, y1, x2, y2]
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union = box1_area + box2_area - intersection
    
    return intersection / union if union > 0 else 0

def apply_nms(df, iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression to bounding boxes in the dataframe
    
    Parameters:
    df: pandas DataFrame containing proposal_bbox and predictedlabel columns
    iou_threshold: threshold for IoU to consider boxes as overlapping
    
    Returns:
    DataFrame with redundant boxes removed
    """
    result_indices = []
    
    # Group by img_id to process each image separately
    for img_id in df['img_id'].unique():
        img_df = df[df['img_id'] == img_id].copy()

        print(img_df)
        
    #     # Sort by confidence (predictedlabel)
    #     img_df = img_df.sort_values('predictedlabel', ascending=False)
        
    #     # Convert string representation of bbox to list if necessary
    #     img_df['bbox'] = img_df['proposal_bbox'].apply(lambda x: 
    #         eval(x) if isinstance(x, str) else x)
        
    #     kept_indices = []
    #     boxes = img_df['bbox'].tolist()
    #     scores = img_df['predictedlabel'].tolist()
    #     indices = range(len(boxes))
        
    #     while indices:
    #         idx = indices[0]
    #         kept_indices.append(img_df.index[idx])
            
    #         overlapping_indices = []
    #         for compare_idx in indices[1:]:
    #             iou = calculate_iou(boxes[idx], boxes[compare_idx])
    #             if iou > iou_threshold:
    #                 overlapping_indices.append(compare_idx)
            
    #         indices = [i for i in indices[1:] if i not in overlapping_indices]
        
    #     result_indices.extend(kept_indices)
    
    # return df.loc[result_indices].sort_index()


apply_nms(merged_df)